# **Assignment 4: some database operations**
- **you will learn:** using joins, aggregation, groupby in Pandas
- **task:**  See section 4.2 below
- **deadline:** 17.11.2025
- [Pandas documentation](https://pandas.pydata.org/docs/index.html)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.

In [9]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

NumPy version: 2.3.3
Pandas version: 2.3.3


In [10]:
# -------------------------------
# 1) Build synthetic datasets
# -------------------------------
# sensor readouts (long-format)
np.random.seed(42)
N_SENSORS = 6
sensors = [f'S{i+1}' for i in range(N_SENSORS)]

start = pd.Timestamp('2025-11-01 00:00')
periods = 24 * 6 # six measurements per hour for 24 hours
freq = '10min' # 10-minute frequency
idx = pd.date_range(start=start, periods=periods, freq=freq)

rows = []
for sensor in sensors:
    base_temp = 15 + 5 * np.random.rand() # baseline temp different per sensor
    base_hum = 40 + 20 * np.random.rand()
    noise_t = np.random.normal(0, 1.5, size=len(idx))
    noise_h = np.random.normal(0, 5.0, size=len(idx))
    # add a diurnal temperature variation
    hours = (idx.hour + idx.minute/60.0)
    diurnal = 6 * np.sin(2 * np.pi * (hours - 6) / 24)
    
    temps = base_temp + diurnal + noise_t
    hums = np.clip(base_hum - 0.5 * diurnal + noise_h, 0, 100)
    
    for ts, t, h in zip(idx, temps, hums):
        rows.append({'sensor_id': sensor, 'timestamp': ts, 'temperature': t, 'humidity': h})

sensor_df = pd.DataFrame(rows)

# add some missing values intentionally and some outliers
sensor_df.loc[sensor_df.sample(frac=0.01, random_state=1).index, 'temperature'] = np.nan
sensor_df.loc[sensor_df.sample(frac=0.005, random_state=2).index, 'humidity'] = np.nan
# inject outliers
outlier_idx = sensor_df.sample(frac=0.002, random_state=3).index
sensor_df.loc[outlier_idx, 'temperature'] += np.random.choice([20, -15], size=len(outlier_idx))

print('--- sensor_df (head) ---')
print(sensor_df.head())

# sensor metadata (wide)
meta = pd.DataFrame({
'sensor_id': sensors,
'location': ['Room A', 'Room B', 'Room C', 'Room A', 'Room D', 'Room B'],
'type': ['thermo-hygro', 'thermo-hygro', 'thermo', 'thermo-hygro', 'hygro', 'thermo']
})

# events table (sparse)
events = pd.DataFrame([
{'sensor_id': 'S1', 'timestamp': start + pd.Timedelta(hours=5), 'event': 'maintenance'},
{'sensor_id': 'S3', 'timestamp': start + pd.Timedelta(hours=8, minutes=20), 'event': 'calibration'},
{'sensor_id': 'S2', 'timestamp': start + pd.Timedelta(hours=18), 'event': 'spike_detected'}
])

--- sensor_df (head) ---
  sensor_id           timestamp  temperature   humidity
0        S1 2025-11-01 00:00:00    11.844233  55.829533
1        S1 2025-11-01 00:10:00    13.162956  55.409148
2        S1 2025-11-01 00:20:00    10.544302  64.612578
3        S1 2025-11-01 00:30:00          NaN  63.473544
4        S1 2025-11-01 00:40:00    13.332673  63.221174


In [6]:
# -------------------------------
# 2) MultiIndex: set index as (sensor_id, timestamp)
# -------------------------------
sensor_mi = sensor_df.set_index(['sensor_id', 'timestamp']).sort_index()
print('\n--- sensor_mi (index sample) ---')
print(sensor_mi.head())

# show selecting using MultiIndex
print('\nSelect all readings for S2 between two times:')
print(sensor_mi.loc['S2'].between_time('06:00', '09:00').head())


--- sensor_mi (index sample) ---
                               temperature   humidity
sensor_id timestamp                                  
S1        2025-11-01 00:00:00    11.844233  55.829533
          2025-11-01 00:10:00    13.162956  55.409148
          2025-11-01 00:20:00    10.544302  64.612578
          2025-11-01 00:30:00          NaN  63.473544
          2025-11-01 00:40:00    13.332673  63.221174

Select all readings for S2 between two times:
                     temperature   humidity
timestamp                                  
2025-11-01 06:00:00    16.292123  38.803557
2025-11-01 06:10:00    16.560178  53.083026
2025-11-01 06:20:00    15.860508  54.342884
2025-11-01 06:30:00    14.793219  44.742754
2025-11-01 06:40:00    16.511539  48.741353


In [11]:
# -------------------------------
# 3) Multi-level columns example: create aggregated summary with MultiIndex columns
# -------------------------------
agg_funcs = {
'temperature': ['mean', 'std', 'min', 'max'],
'humidity': ['mean', 'std']
}
summary_by_sensor = sensor_df.groupby('sensor_id').agg(agg_funcs)
# flatten/unflatten demonstration: keep as MultiIndex columns
print('\n--- summary_by_sensor (MultiIndex columns) ---')
print(summary_by_sensor.head())

# rename columns to readable MultiIndex (level names)
summary_by_sensor.columns.names = ['measurement', 'stat']
print('\nColumns levels:', summary_by_sensor.columns.names)

# access a specific subtable using xs
print('\nTemperature means:')
print(summary_by_sensor.xs('temperature', axis=1)[['mean', 'std']])


--- summary_by_sensor (MultiIndex columns) ---
          temperature                                  humidity          
                 mean       std       min        max       mean       std
sensor_id                                                                
S1          16.718369  4.511676  8.552629  25.213955  59.259889  5.560005
S2          15.427998  4.497470  2.122613  24.023564  45.599224  5.493384
S3          15.658394  4.421943  7.273332  25.556400  56.385015  5.146519
S4          15.668049  4.238744  7.365955  24.204889  48.812680  5.493305
S5          17.801163  4.889507  9.133021  38.594542  49.913718  5.367881

Columns levels: ['measurement', 'stat']

Temperature means:
stat            mean       std
sensor_id                     
S1         16.718369  4.511676
S2         15.427998  4.497470
S3         15.658394  4.421943
S4         15.668049  4.238744
S5         17.801163  4.889507
S6         18.627710  4.633420


In [8]:
summary_by_sensor.columns = [
    f"{measurement}_{stat}"
    for measurement, stat in summary_by_sensor.columns
]

print('\n--- summary_by_sensor (flat columns) ---')
print(summary_by_sensor.head())


--- summary_by_sensor (flat columns) ---
           temperature_mean  temperature_std  temperature_min  \
sensor_id                                                       
S1                16.718369         4.511676         8.552629   
S2                15.427998         4.497470         2.122613   
S3                15.658394         4.421943         7.273332   
S4                15.668049         4.238744         7.365955   
S5                17.801163         4.889507         9.133021   

           temperature_max  humidity_mean  humidity_std  
sensor_id                                                
S1               25.213955      59.259889      5.560005  
S2               24.023564      45.599224      5.493384  
S3               25.556400      56.385015      5.146519  
S4               24.204889      48.812680      5.493305  
S5               38.594542      49.913718      5.367881  


In [12]:
# -------------------------------
# 4) Advanced groupby with named aggregation and custom functions
# -------------------------------
# We'll compute per-sensor and per-location aggregations using named aggregations
sensor_with_meta = sensor_df.merge(meta, on='sensor_id', how='left')
## iny sposob: sensor_with_meta = sensor_df.merge(meta, left_on='sensor_id', right_on = "sensor_id" how='left')
agg_named = sensor_with_meta.groupby(['location']).agg(
temp_mean=('temperature', 'mean'),
temp_med=('temperature', 'median'),
temp_iqr=('temperature', lambda x: np.subtract(*np.percentile(x.dropna(), [75, 25]))),
humidity_mean=('humidity', 'mean'),
n_readings=('temperature', 'count')
)
print('\n--- agg_named by location ---')
print(agg_named)

# custom aggregation using apply (slower but expressive) -> compute time-of-day sensitivity

def day_night_diff(group):
    group = group.copy()
    # daytime mean vs nighttime mean
    group['hour'] = group['timestamp'].dt.hour
    daytime = group.loc[(group['hour'] >= 6) & (group['hour'] < 18), 'temperature']
    nighttime = group.loc[(group['hour'] < 6) | (group['hour'] >= 18), 'temperature']
    return pd.Series({'day_minus_night': daytime.mean() - nighttime.mean()})

sens_daynight = sensor_df.merge(meta, on='sensor_id').groupby('sensor_id').apply(day_night_diff, include_groups=False)
print('\n--- day_minus_night per sensor ---')
print(sens_daynight.head())


--- agg_named by location ---
          temp_mean   temp_med  temp_iqr  humidity_mean  n_readings
location                                                           
Room A    16.187642  16.260233  7.800304      54.036285         283
Room B    17.022280  16.570175  7.810141      48.698361         287
Room C    15.658394  15.917286  8.167460      56.385015         143
Room D    17.801163  18.022922  8.284296      49.913718         142

--- day_minus_night per sensor ---
           day_minus_night
sensor_id                 
S1                7.691988
S2                7.222448
S3                7.461552
S4                7.165945
S5                7.980190


In [ ]:
# Note on merge vs join
# pandas.merge() is the core function for all merges/joins.
# df.merge() and df.join() are convenient wrappers:
# - df.join() defaults to left join on df1's index and df2's index.
# - df.merge() defaults to inner join on columns, but can join on indexes.
# Use merge() for flexibility; join() can save typing in simple index-based left joins.

In [10]:
# -------------------------------
# 5) pivot_table & crosstab
# -------------------------------
# pivot_table: average temperature per sensor per hour of day
sensor_df['hour'] = sensor_df['timestamp'].dt.hour
pv = pd.pivot_table(sensor_df, values='temperature', index='sensor_id', columns='hour', aggfunc='mean')
print('\n--- pivot_table mean temperature by sensor x hour ---')
print(pv.iloc[:, :6]) # show first 6 hours as sample

# crosstab: count of readings per sensor per type
sensor_meta_joined = sensor_df.merge(meta, on='sensor_id')
ct = pd.crosstab(sensor_meta_joined['sensor_id'], sensor_meta_joined['type'])
print('\n--- crosstab counts sensor x type ---')
print(ct)


--- pivot_table mean temperature by sensor x hour ---
hour               0          1          2          3          4          5
sensor_id                                                                  
S1         12.210048  10.664322  10.720927  12.989877  14.358660  14.856285
S2          9.690962  10.132154  10.812555  12.572447  13.260633  14.696669
S3         10.254658  10.215027  10.674089  13.038813  12.860999  15.427306
S4          9.326482  10.789089  11.722838  13.088461  13.111283  14.757508
S5         12.241501  12.651350  11.540636  13.750657  14.472117  16.459888
S6         13.411637  12.972633  14.169101  15.001611  16.085027  17.580157

--- crosstab counts sensor x type ---
type       hygro  thermo  thermo-hygro
sensor_id                             
S1             0       0           144
S2             0       0           144
S3             0     144             0
S4             0       0           144
S5           144       0             0
S6             0     144 

In [11]:
# -------------------------------
# 6) Merging and joining examples (all possibilities)
# -------------------------------
# inner merge (only sensors present in both)
m_inner = sensor_df.merge(meta, on='sensor_id', how='inner')
# left merge (keeps all readings)
m_left = sensor_df.merge(meta, on='sensor_id', how='left')
# right and outer
m_right = sensor_df.merge(meta, on='sensor_id', how='right')
m_outer = sensor_df.merge(meta, on='sensor_id', how='outer')

print('\nmerge sizes: inner={}, left={}, right={}, outer={}'.format(
len(m_inner), len(m_left), len(m_right), len(m_outer)
))

# join on index: prepare two DataFrames with indexes
df_a = sensor_df.sample(50, random_state=4).set_index(['sensor_id', 'timestamp']).sort_index()
df_b = pd.DataFrame({'battery': np.random.randint(20, 100, size=30)},
index=df_a.index[:30])
# left join using index
joined = df_a.join(df_b, how='left')

print('\n--- joined (index join) sample ---')
print(joined.head())

# concat: stack vertically and horizontally
concat_v = pd.concat([sensor_df.head(5), sensor_df.tail(5)], axis=0)
concat_h = pd.concat([sensor_df.head(5).reset_index(drop=True).iloc[:, :3],
sensor_df.head(5).reset_index(drop=True).iloc[:, 3:]], axis=1)

# combine_first: combine two sources preferring left's non-null values
left = sensor_df.head(10).copy()
right = left.copy()
right.loc[right.sample(frac=0.3, random_state=5).index, 'temperature'] = np.nan
combined = right.combine_first(left)
print('\n--- combine_first example ---')
print(combined.head())


merge sizes: inner=864, left=864, right=864, outer=864

--- joined (index join) sample ---
                               temperature   humidity  hour  battery
sensor_id timestamp                                                 
S1        2025-11-01 07:50:00    16.998632  51.400346     7     74.0
          2025-11-01 10:30:00    24.450338  75.506305    10     35.0
          2025-11-01 11:00:00    23.210710  60.886517    11     81.0
          2025-11-01 11:10:00    21.762797  59.342354    11     67.0
          2025-11-01 11:40:00    22.796130  52.161576    11     49.0

--- combine_first example ---
  sensor_id           timestamp  temperature   humidity  hour
0        S1 2025-11-01 00:00:00    11.844233  55.829533     0
1        S1 2025-11-01 00:10:00    13.162956  55.409148     0
2        S1 2025-11-01 00:20:00    10.544302  64.612578     0
3        S1 2025-11-01 00:30:00          NaN  63.473544     0
4        S1 2025-11-01 00:40:00    13.332673  63.221174     0


In [ ]:
# -------------------------------
# 7) Reshaping: stack/unstack and melt
# -------------------------------
# create a small pivot and show stack/unstack
small = sensor_df[sensor_df['sensor_id'].isin(['S1', 'S2'])].head(12)
pv_small = small.pivot(index='timestamp', columns='sensor_id', values='temperature')
print('\n--- pv_small pivot (timestamp x sensor) ---')
print(pv_small.head())
print('\nstacked -> unstacked roundtrip (demonstration)')
stacked = pv_small.stack()
print(stacked.head())
print(stacked.unstack().head())

# melt: convert wide back to long
melted = pv_small.reset_index().melt(id_vars='timestamp', var_name='sensor_id', value_name='temperature')
print('\n--- melted back to long ---')
print(melted.head())

In [12]:
# -------------------------------
# 8) Query: expressive filtering
# -------------------------------
# Use DataFrame.query for readable boolean expressions
q = sensor_df.query('temperature > 25 and humidity < 50')
print(f'\nNumber of readings with temperature>25 and humidity<50: {len(q)}')

# boolean indexing vs query for more complex expressions
expr = "(temperature > temperature.mean()) & (humidity < humidity.quantile(0.25))"
# careful: can't use temperature.mean() inside query easily; compute first
temp_mean = sensor_df['temperature'].mean()
hum_q25 = sensor_df['humidity'].quantile(0.25)
q2 = sensor_df.query('@temp_mean < temperature and humidity < @hum_q25')
print(f'\nUsing external variables in query, matches: {len(q2)}')


Number of readings with temperature>25 and humidity<50: 12

Using external variables in query, matches: 139


In [16]:
# -------------------------------
# 9) Aggregations with groupby + transform and filter
# -------------------------------
# find sensors whose median temperature is > global median
global_median = sensor_df['temperature'].median()
sensor_medians = sensor_df.groupby('sensor_id')['temperature'].median()
hot_sensors = sensor_medians[sensor_medians > global_median].index.tolist()
print('\nSensors with median temperature > global median:', hot_sensors)

# use transform to broadcast group statistic back to rows
sensor_df['sensor_temp_median'] = sensor_df.groupby('sensor_id')['temperature'].transform('median')
# filter rows where temperature is above group's median
above_group_median = sensor_df[sensor_df['temperature'] > sensor_df['sensor_temp_median']]
print('\nRows above their sensor median (sample):')
print(above_group_median.head())

# groupby.filter to keep only sensors with at least 200 readings
sensors_with_enough = sensor_df.groupby('sensor_id').filter(lambda g: len(g) >= 200)
print('\nSensors kept (len>=200):', sensors_with_enough['sensor_id'].unique())


Sensors with median temperature > global median: ['S1', 'S5', 'S6']

Rows above their sensor median (sample):
   sensor_id           timestamp  temperature   humidity  hour  \
29        S1 2025-11-01 04:50:00    17.846883  64.052320     4   
32        S1 2025-11-01 05:20:00    17.064629  58.211946     5   
37        S1 2025-11-01 06:10:00    17.429709  61.295790     6   
38        S1 2025-11-01 06:20:00    18.503335  57.635505     6   
39        S1 2025-11-01 06:30:00    17.912910  62.192710     6   

    sensor_temp_median  
29            17.01069  
32            17.01069  
37            17.01069  
38            17.01069  
39            17.01069  

Sensors kept (len>=200): []


In [13]:
# -------------------------------
# 10) MultiIndex columns: produce aggregated table with hierarchical columns and plot
# -------------------------------
agg_multi = sensor_df.groupby('sensor_id').agg(
temp_mean=('temperature', 'mean'),
temp_std=('temperature', 'std'),
hum_mean=('humidity', 'mean'),
hum_std=('humidity', 'std')
)
# create MultiIndex columns manually
agg_multi.columns = pd.MultiIndex.from_tuples([('temperature', 'mean'), ('temperature', 'std'), ('humidity', 'mean'), ('humidity', 'std')])
print('\n--- agg_multi (MultiIndex columns) ---')
print(agg_multi)


--- agg_multi (MultiIndex columns) ---
          temperature             humidity          
                 mean       std       mean       std
sensor_id                                           
S1          16.718369  4.511676  59.259889  5.560005
S2          15.427998  4.497470  45.599224  5.493384
S3          15.658394  4.421943  56.385015  5.146519
S4          15.668049  4.238744  48.812680  5.493305
S5          17.801163  4.889507  49.913718  5.367881
S6          18.627710  4.633420  51.732933  5.117629


In [17]:
# -------------------------------
# 11) Rolling, expanding and time-based resampling (for sensor S2)
# -------------------------------
## je potreba to zoradit
s2 = sensor_df[sensor_df['sensor_id'] == 'S2'].set_index('timestamp').sort_index()
# rolling 30-minute window (freq is 10min so 3 observations -> use time-based window)
s2['temp_roll_30min'] = s2['temperature'].rolling('30min').mean()
# humidity diff next reading -> shift(-1)
s2['humidity_next_diff'] = s2['humidity'].shift(-1) - s2['humidity']
# boolean: temp increased >5C and humidity dropped >20 compared to previous measurement
s2['temp_prev_diff'] = s2['temperature'] - s2['temperature'].shift(1)
s2['hum_prev_diff'] = s2['humidity'].shift(1) - s2['humidity']
s2['extreme_change'] = (s2['temp_prev_diff'] > 5) & (s2['hum_prev_diff'] > 20)

## Task Specification: Aggregations, Joins, GroupBy, and Rolling Analysis with Pandas

---

### **Task Overview**
This assignment simulates a **customer purchase analytics workflow** using Pandas. You will work with timestamped transaction data and customer metadata, performing:

- Group-based aggregations
- Time-based analysis
- Rolling window calculations within groups
- Joining customer metadata from another dataset

---

### **Datasets**
You are provided with two datasets:

| File | Description |
|------|-------------|
| `data/purchases.csv` | Timestamped purchase data (contains `Customer_id`, `Timestamp`, `Price`, etc.) |
| `data/customer_info.csv` | Customer metadata (contains `Customer_id`, `Born_date`, `City`, etc.) |

---

### **1. Load and Prepare the Data**
- **Task 1.1:** Load the two CSV files into Pandas DataFrames:
  - Use `df` for `purchases.csv`
  - Use `customer_info` for `customer_info.csv`

- **Task 1.2:** Convert the `Timestamp` column in `df` to Pandas `datetime` format.

- **Task 1.3:** Set `Timestamp` as the **index** of `df` and **sort** the index in ascending order.

---

### **2. Join Customer Metadata**
- **Task 2.1:** Merge the purchase DataFrame (`df`) with the customer metadata DataFrame (`customer_info`) on the `Customer_id` column.

- **Task 2.2:** The resulting DataFrame should include the columns `Born_date` and `City` from the `customer_info` DataFrame.

---

### **3. GroupBy and Aggregations**
Using the merged DataFrame:

- **Task 3.1:** Group by **Customer_id** and compute:
  - Total revenue (`sum(Price)`)
  - Number of purchases (`count`)
  - Average purchase price (`mean(Price)`)

- **Task 3.2:** Create a new feature by extracting the **month name** (e.g., `"January"`, `"February"`) from the `Timestamp`.

- **Task 3.3:** Group by **City** and **Month** and compute:
  - Total monthly revenue per city (`sum(Price)`)

---

### **4. Rolling Window Analysis (Per Customer)**
Perform rolling window calculations **within each customer group**:

- Inside each customer group:
  - **Sort** the DataFrame by `Timestamp`.

- Compute the following:
  - A **3-purchase rolling average** of `Price`.
  - A `diff()` column showing the change in price compared to the previous purchase.

> **Note:** The rolling operations must be computed **within each customer group**, not across all customers.

---

### **5. Advanced Filtering**
- Do the same as in 4 only for customers from **Berlin**, make sure only necessary values are joined (that is filter first, join second).

- Do the same as in 4 only for rows where the `Price` of the current purchase is greater than **20%** than the previous purchase (`Price`), make sure only necessary values are joined (that is filter first, join second).

---

### **6. Custom Lambda Function for GroupBy Operations**
- Apply a **custom lambda function** within a `groupby()` operation to filter or modify the data. For example:
  - Use a lambda to **filter** customers who have purchased more than a specific amount within a given timeframe (e.g., greater than $1000 in total purchases during the last quarter).
  - Apply custom transformations to grouped data using the lambda function.

---

### 7. Time-Based Analysis
- Perform additional time-based analysis:
  - Calculate **monthly growth rate** for total revenue per customer (i.e., compare total revenue of the current month with the previous month).
  - Create a **cumulative sum** of purchases over time per customer.


In [1]:
import pandas as pd

In [13]:
## load the data
data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/data.csv')
df = pd.read_csv("data/purchases.csv")
customer_info = pd.read_csv("data/customer_info.csv")

In [32]:
## set timestamp to date format
df.Timestamp = pd.to_datetime(df.Timestamp)
print(df.Timestamp.dtypes)

datetime64[ns]


In [110]:
## set timestamp in df (purchases) as the index and sort it
data_index = df.set_index( 'Timestamp').sort_index()
print('\n--- data_index (index sample) ---')
print(data_index.head())


--- data_index (index sample) ---
                    Customer_id   Price
Timestamp                              
2025-01-01 00:00:00        C037  436.71
2025-01-01 00:30:00        C049  274.70
2025-01-01 01:00:00        C017  380.42
2025-01-01 01:30:00        C049  378.28
2025-01-01 02:00:00        C002   13.88


In [111]:
df_merged_with_cinfo= df.merge(customer_info, on='Customer_id', how='left')
print(df_merged_with_cinfo.head())

  Customer_id           Timestamp   Price   Born_date       City
0        C037 2025-01-01 00:00:00  436.71  1996-09-14     Munich
1        C049 2025-01-01 00:30:00  274.70  1979-01-20     Munich
2        C017 2025-01-01 01:00:00  380.42  1981-02-23    Cologne
3        C049 2025-01-01 01:30:00  378.28  1979-01-20     Munich
4        C002 2025-01-01 02:00:00   13.88  1988-07-03  Frankfurt


In [112]:
## 3 GroupBy and Aggregations
# group by customer id
df_by_id = df_merged_with_cinfo.groupby(['Customer_id']).agg(
total_revenue=('Price', "sum"),
number_of_purchases=('Price', "count"),
avg_price=('Price', "mean"),
)
print('\n--- df_by_id ---')
print(df_by_id.head())


--- df_by_id ---
             total_revenue  number_of_purchases   avg_price
Customer_id                                                
C001              13055.75                   56  233.138393
C002              15927.80                   64  248.871875
C003              14947.34                   56  266.916786
C004              15111.43                   59  256.125932
C005              16798.61                   64  262.478281


In [114]:
## 3.2:** Create a new feature by extracting the **month name** (e.g., `"January"`, `"February"`) from the `Timestamp`.
df_merged_with_cinfo["month"] = df_merged_with_cinfo["Timestamp"].dt.month_name()
print("Dataframe with months:")
print(df_merged_with_cinfo.head())


Dataframe with months:
  Customer_id           Timestamp   Price   Born_date       City    month
0        C037 2025-01-01 00:00:00  436.71  1996-09-14     Munich  January
1        C049 2025-01-01 00:30:00  274.70  1979-01-20     Munich  January
2        C017 2025-01-01 01:00:00  380.42  1981-02-23    Cologne  January
3        C049 2025-01-01 01:30:00  378.28  1979-01-20     Munich  January
4        C002 2025-01-01 02:00:00   13.88  1988-07-03  Frankfurt  January


In [115]:
## 3.3:** Group by **City** and **Month** and compute:
 # - Total monthly revenue per city (`sum(Price)`)
df_groupedby_city_and_month = (df_merged_with_cinfo.groupby(['City', 'month'], as_index=False)['Price'].sum())
print("Dataframe grouped by city and month:")
print(df_groupedby_city_and_month.head(10))

Dataframe grouped by city and month:
        City     month      Price
0     Berlin  February   86163.85
1     Berlin   January  101192.17
2     Berlin     March   10474.86
3    Cologne  February   86781.31
4    Cologne   January   98030.00
5    Cologne     March   11220.64
6  Frankfurt  February   52020.01
7  Frankfurt   January   61492.29
8  Frankfurt     March    6698.88
9    Hamburg  February   42056.17


In [59]:
## 4 Rolling Window Analysis (Per Customer)
## sort by timestamp
df_sorted = df_merged_with_cinfo.sort_values(['Customer_id', 'Timestamp'])
print(df_sorted.head(10))


    Customer_id           Timestamp   Price   Born_date       City    month
11         C001 2025-01-01 05:30:00  376.61  1998-12-03  Frankfurt  January
52         C001 2025-01-02 02:00:00   63.23  1998-12-03  Frankfurt  January
68         C001 2025-01-02 10:00:00  108.83  1998-12-03  Frankfurt  January
202        C001 2025-01-05 05:00:00   70.19  1998-12-03  Frankfurt  January
254        C001 2025-01-06 07:00:00  132.80  1998-12-03  Frankfurt  January
255        C001 2025-01-06 07:30:00  338.93  1998-12-03  Frankfurt  January
263        C001 2025-01-06 11:30:00  415.65  1998-12-03  Frankfurt  January
304        C001 2025-01-07 08:00:00  196.76  1998-12-03  Frankfurt  January
327        C001 2025-01-07 19:30:00  382.14  1998-12-03  Frankfurt  January
453        C001 2025-01-10 10:30:00  265.98  1998-12-03  Frankfurt  January


In [60]:
# 3-purchase rolling average using groupby().apply()
df_rolling = (
    df_sorted
    .groupby('Customer_id', group_keys=False)  # avoid adding Customer_id as extra index
    .apply(lambda x: x.assign(
        Price_RollingAvg3=x['Price'].rolling(window=3, min_periods=1).mean()
    ))
)
print(df_rolling[['Customer_id', 'Timestamp', 'Price', 'Price_RollingAvg3']].head(10))


    Customer_id           Timestamp   Price  Price_RollingAvg3
11         C001 2025-01-01 05:30:00  376.61         376.610000
52         C001 2025-01-02 02:00:00   63.23         219.920000
68         C001 2025-01-02 10:00:00  108.83         182.890000
202        C001 2025-01-05 05:00:00   70.19          80.750000
254        C001 2025-01-06 07:00:00  132.80         103.940000
255        C001 2025-01-06 07:30:00  338.93         180.640000
263        C001 2025-01-06 11:30:00  415.65         295.793333
304        C001 2025-01-07 08:00:00  196.76         317.113333
327        C001 2025-01-07 19:30:00  382.14         331.516667
453        C001 2025-01-10 10:30:00  265.98         281.626667


C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\2415866097.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [116]:
# diff of Price within each customer group
df_final = (
    df_rolling
    .groupby('Customer_id', group_keys=False)  # keep Customer in columns, not index
    .apply(lambda x: x.assign(
        Price_Diff=x['Price'].diff()
    ))
)
print(df_final.head(10))


    Customer_id           Timestamp   Price   Born_date       City    month  \
11         C001 2025-01-01 05:30:00  376.61  1998-12-03  Frankfurt  January   
52         C001 2025-01-02 02:00:00   63.23  1998-12-03  Frankfurt  January   
68         C001 2025-01-02 10:00:00  108.83  1998-12-03  Frankfurt  January   
202        C001 2025-01-05 05:00:00   70.19  1998-12-03  Frankfurt  January   
254        C001 2025-01-06 07:00:00  132.80  1998-12-03  Frankfurt  January   
255        C001 2025-01-06 07:30:00  338.93  1998-12-03  Frankfurt  January   
263        C001 2025-01-06 11:30:00  415.65  1998-12-03  Frankfurt  January   
304        C001 2025-01-07 08:00:00  196.76  1998-12-03  Frankfurt  January   
327        C001 2025-01-07 19:30:00  382.14  1998-12-03  Frankfurt  January   
453        C001 2025-01-10 10:30:00  265.98  1998-12-03  Frankfurt  January   

     Price_RollingAvg3  Price_Diff  
11          376.610000         NaN  
52          219.920000     -313.38  
68          182.890

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\1593796692.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [63]:
## customer info with city = Berlin.
df_berlin = df_merged_with_cinfo[df_merged_with_cinfo['City'] == 'Berlin']

In [65]:
## sort by timestamp
df_berlin_sorted = df_berlin.sort_values(["Customer_id", "Timestamp"])
print(df_berlin_sorted.head(10))

    Customer_id           Timestamp   Price   Born_date    City    month
197        C013 2025-01-05 02:30:00  474.31  1995-06-12  Berlin  January
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January
438        C013 2025-01-10 03:00:00   34.44  1995-06-12  Berlin  January
473        C013 2025-01-10 20:30:00  405.55  1995-06-12  Berlin  January
483        C013 2025-01-11 01:30:00  276.82  1995-06-12  Berlin  January
541        C013 2025-01-12 06:30:00  448.38  1995-06-12  Berlin  January
545        C013 2025-01-12 08:30:00  470.52  1995-06-12  Berlin  January


In [66]:
# 3-purchase rolling average using groupby().apply()
df_berlin_rolling = (
    df_berlin_sorted
    .groupby("Customer_id", group_keys=False)
    .apply(lambda x: x.assign(
        Rolling_berlin = x['Price'].rolling(window=3, min_periods=1).mean()
    ))
)
print(df_berlin_rolling.head(10))


    Customer_id           Timestamp   Price   Born_date    City    month  \
197        C013 2025-01-05 02:30:00  474.31  1995-06-12  Berlin  January   
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January   
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January   
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January   
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January   
438        C013 2025-01-10 03:00:00   34.44  1995-06-12  Berlin  January   
473        C013 2025-01-10 20:30:00  405.55  1995-06-12  Berlin  January   
483        C013 2025-01-11 01:30:00  276.82  1995-06-12  Berlin  January   
541        C013 2025-01-12 06:30:00  448.38  1995-06-12  Berlin  January   
545        C013 2025-01-12 08:30:00  470.52  1995-06-12  Berlin  January   

     Rolling_berlin  
197      474.310000  
240      442.160000  
334      427.073333  
380      299.870000  
392      293.960000  
438      173.140000  
473      

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\3773206787.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [68]:
# difference:
df_berlin_final =(
    df_berlin_rolling
    .groupby('Customer_id', group_keys=False)
    .apply(lambda x: x.assign(
        Diff_berlin=x['Price'].diff()
    ))
)
print(df_berlin_final.head(10))

    Customer_id           Timestamp   Price   Born_date    City    month  \
197        C013 2025-01-05 02:30:00  474.31  1995-06-12  Berlin  January   
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January   
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January   
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January   
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January   
438        C013 2025-01-10 03:00:00   34.44  1995-06-12  Berlin  January   
473        C013 2025-01-10 20:30:00  405.55  1995-06-12  Berlin  January   
483        C013 2025-01-11 01:30:00  276.82  1995-06-12  Berlin  January   
541        C013 2025-01-12 06:30:00  448.38  1995-06-12  Berlin  January   
545        C013 2025-01-12 08:30:00  470.52  1995-06-12  Berlin  January   

     Rolling_berlin  Diff_berlin  
197      474.310000          NaN  
240      442.160000       -64.30  
334      427.073333       -13.11  
380      299.870000    

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\3069820236.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [117]:
## 5-druhá časť
# použijeme df_berlin_sorted a ten vyfiltrujeme
df_berlin_sorted['Prev_Price'] = df_berlin_sorted.groupby('Customer_id')['Price'].shift(1)
print(df_berlin_sorted.head())

    Customer_id           Timestamp   Price   Born_date    City    month  \
197        C013 2025-01-05 02:30:00  474.31  1995-06-12  Berlin  January   
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January   
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January   
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January   
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January   

     Prev_Price  
197         NaN  
240      474.31  
334      410.01  
380      396.90  
392       92.70  


In [118]:
## vyfiltrujeme
df_berlin_filtered = df_berlin_sorted[df_berlin_sorted['Price'] > 0.2 * df_berlin_sorted['Prev_Price']]

    Customer_id           Timestamp   Price   Born_date    City    month  \
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January   
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January   
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January   
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January   
473        C013 2025-01-10 20:30:00  405.55  1995-06-12  Berlin  January   

     Prev_Price  
240      474.31  
334      410.01  
380      396.90  
392       92.70  
473       34.44  


In [76]:
## rolling
df_berlin_cond_rolling = (df_berlin_filtered
    .groupby(['Customer_id'], group_keys=False)
    .apply(lambda x: x.assign(
        Rolling_berlin_filtered = x['Price'].rolling(window=3, min_periods=1).mean()
    ))
)
print(df_berlin_rolling.head(10))

    Customer_id           Timestamp   Price   Born_date    City    month  \
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January   
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January   
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January   
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January   
473        C013 2025-01-10 20:30:00  405.55  1995-06-12  Berlin  January   
483        C013 2025-01-11 01:30:00  276.82  1995-06-12  Berlin  January   
541        C013 2025-01-12 06:30:00  448.38  1995-06-12  Berlin  January   
545        C013 2025-01-12 08:30:00  470.52  1995-06-12  Berlin  January   
572        C013 2025-01-12 22:00:00  472.27  1995-06-12  Berlin  January   
601        C013 2025-01-13 12:30:00  369.86  1995-06-12  Berlin  January   

     Prev_Price  Rolling_berlin_filtered  
240      474.31               410.010000  
334      410.01               403.455000  
380      396.90               299.

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\2584334059.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [78]:
## difference
df_berlin_cond_final = (df_berlin_cond_rolling
     .groupby(['Customer_id'], group_keys=False)
     .apply(lambda x: x.assign(
    Diff_berlin_cond=x['Price'].diff()
))
)
print(df_berlin_cond_final.head(10))

    Customer_id           Timestamp   Price   Born_date    City    month  \
240        C013 2025-01-06 00:00:00  410.01  1995-06-12  Berlin  January   
334        C013 2025-01-07 23:00:00  396.90  1995-06-12  Berlin  January   
380        C013 2025-01-08 22:00:00   92.70  1995-06-12  Berlin  January   
392        C013 2025-01-09 04:00:00  392.28  1995-06-12  Berlin  January   
473        C013 2025-01-10 20:30:00  405.55  1995-06-12  Berlin  January   
483        C013 2025-01-11 01:30:00  276.82  1995-06-12  Berlin  January   
541        C013 2025-01-12 06:30:00  448.38  1995-06-12  Berlin  January   
545        C013 2025-01-12 08:30:00  470.52  1995-06-12  Berlin  January   
572        C013 2025-01-12 22:00:00  472.27  1995-06-12  Berlin  January   
601        C013 2025-01-13 12:30:00  369.86  1995-06-12  Berlin  January   

     Prev_Price  Rolling_berlin_filtered  Diff_berlin_cond  
240      474.31               410.010000               NaN  
334      410.01               403.455000 

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\2197106767.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [119]:
## 6 Custom lambda function for GroupBy operations
## we will use the dataframe df_merged_with_cinfo
custom_lambda = (
    df_merged_with_cinfo
    .groupby('Customer_id', group_keys=False)
    .filter(lambda x: x['Price'].sum() > 1000)
)
print(custom_lambda.head(10))

  Customer_id           Timestamp   Price   Born_date       City    month
0        C037 2025-01-01 00:00:00  436.71  1996-09-14     Munich  January
1        C049 2025-01-01 00:30:00  274.70  1979-01-20     Munich  January
2        C017 2025-01-01 01:00:00  380.42  1981-02-23    Cologne  January
3        C049 2025-01-01 01:30:00  378.28  1979-01-20     Munich  January
4        C002 2025-01-01 02:00:00   13.88  1988-07-03  Frankfurt  January
5        C002 2025-01-01 02:30:00   67.50  1988-07-03  Frankfurt  January
6        C028 2025-01-01 03:00:00  331.52  2003-05-03     Berlin  January
7        C023 2025-01-01 03:30:00   85.00  1961-08-08     Berlin  January
8        C037 2025-01-01 04:00:00  131.47  1996-09-14     Munich  January
9        C032 2025-01-01 04:30:00  473.59  1987-10-03     Munich  January


In [122]:
## 7
# vytvoríme si stĺpec, kde bude total revenue:
df_merged_with_cinfo["Total_revenue"] = df_merged_with_cinfo.groupby(["Customer_id", "month"])["Price"].transform("sum")
df_merged_with_cinfo = df_merged_with_cinfo.sort_values(['Customer_id', 'month'])

print(df_merged_with_cinfo.head(10))

     Customer_id           Timestamp   Price   Born_date       City     month  \
1618        C001 2025-02-03 17:00:00  418.36  1998-12-03  Frankfurt  February   
1677        C001 2025-02-04 22:30:00  233.88  1998-12-03  Frankfurt  February   
1916        C001 2025-02-09 22:00:00  133.40  1998-12-03  Frankfurt  February   
1935        C001 2025-02-10 07:30:00  455.85  1998-12-03  Frankfurt  February   
1941        C001 2025-02-10 10:30:00   89.79  1998-12-03  Frankfurt  February   
1967        C001 2025-02-10 23:30:00  254.79  1998-12-03  Frankfurt  February   
2002        C001 2025-02-11 17:00:00   71.86  1998-12-03  Frankfurt  February   
2012        C001 2025-02-11 22:00:00  234.71  1998-12-03  Frankfurt  February   
2032        C001 2025-02-12 08:00:00  243.94  1998-12-03  Frankfurt  February   
2072        C001 2025-02-13 04:00:00   68.05  1998-12-03  Frankfurt  February   

      Total_revenue  
1618        5204.79  
1677        5204.79  
1916        5204.79  
1935        5204.79 

In [123]:
## teraz vypočítame growth rate
df_growth_rate =(
    df_merged_with_cinfo
    .groupby('Customer_id', group_keys=False)
    .apply(lambda x: x.assign(
        Growth_rate = (x['Total_revenue'].diff() / x['Total_revenue'].shift(1) ) * 100
    ))
)
print(df_growth_rate.head(20))

     Customer_id           Timestamp   Price   Born_date       City     month  \
1618        C001 2025-02-03 17:00:00  418.36  1998-12-03  Frankfurt  February   
1677        C001 2025-02-04 22:30:00  233.88  1998-12-03  Frankfurt  February   
1916        C001 2025-02-09 22:00:00  133.40  1998-12-03  Frankfurt  February   
1935        C001 2025-02-10 07:30:00  455.85  1998-12-03  Frankfurt  February   
1941        C001 2025-02-10 10:30:00   89.79  1998-12-03  Frankfurt  February   
1967        C001 2025-02-10 23:30:00  254.79  1998-12-03  Frankfurt  February   
2002        C001 2025-02-11 17:00:00   71.86  1998-12-03  Frankfurt  February   
2012        C001 2025-02-11 22:00:00  234.71  1998-12-03  Frankfurt  February   
2032        C001 2025-02-12 08:00:00  243.94  1998-12-03  Frankfurt  February   
2072        C001 2025-02-13 04:00:00   68.05  1998-12-03  Frankfurt  February   
2078        C001 2025-02-13 07:00:00  162.54  1998-12-03  Frankfurt  February   
2186        C001 2025-02-15 

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\1079212698.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(


In [124]:
## cumulative sum
df_sorted = df_merged_with_cinfo.sort_values(['Customer_id', 'month'])
df_cumsum = (
    df_sorted
    .groupby('Customer_id', group_keys=False)
    .apply(lambda x: x.assign(
        Cumulative_Purchases=x['Price'].cumsum()
    ))
)

print(df_cumsum[['Customer_id', 'month', 'Price', 'Cumulative_Purchases']].head(20))



     Customer_id     month   Price  Cumulative_Purchases
1618        C001  February  418.36                418.36
1677        C001  February  233.88                652.24
1916        C001  February  133.40                785.64
1935        C001  February  455.85               1241.49
1941        C001  February   89.79               1331.28
1967        C001  February  254.79               1586.07
2002        C001  February   71.86               1657.93
2012        C001  February  234.71               1892.64
2032        C001  February  243.94               2136.58
2072        C001  February   68.05               2204.63
2078        C001  February  162.54               2367.17
2186        C001  February   87.20               2454.37
2340        C001  February  464.39               2918.76
2367        C001  February  221.34               3140.10
2420        C001  February  239.15               3379.25
2513        C001  February  196.04               3575.29
2550        C001  February  275

C:\Users\Richard\AppData\Local\Temp\ipykernel_20640\4282740404.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(
